# **Lab 1: Matrix factorization**
**Christian Weigelt**

# **Abstract**

This lab consisted of the implementation of a selection of linear algebra functions, and one simple method, as well as some functions related to matrix factorization and the direct solving of linear equation systems. Test code is written to verify the correctness and performance of output. 
  In the introduction section, the functions are given a brief description, both of input/output, and what is to be tested. 
  In the method section, short definitions of the functions are given, and their respective implementation and test function is presented.
  In the results section, the output of the test cases is presented.

# **Set up environment**

In [62]:
import numpy as np
import scipy.sparse as sparse
import math

# **Introduction**

In this lab, the assignment was to implement 4 functions, with input and output as defined in the lab instructions, as well as write code tests to test output.
  
1. Function: sparse matrix-vector product

  Input: vector x, sparse (real, quadratic) matrix A: CRS arrays val, col_idx, row_ptr</br>
  Output: matrix-vector product b=Ax
  
  Test: verify accuracy against dense matrix-vector product. 
</br>
2. Function: QR factorization

  Input: (real quadratic) matrix A</br>
  Output: orthogonal matrix Q, upper triangular matrix R, such that A=QR
  
  Test: R upper triangular, Frobenius norms || Q^TQ-I ||_F, || QR-A ||_F
</br>
3. Function: direct solver Ax=b

  Input: (real, quadratic) matrix A, vector b</br>
  Output: vector x=A^-1b
  
  Test: residual || Ax-b ||, and || x-y || where y is a manufactured solution with b=Ay
</br>
4. Function: least squares problem Ax=b

  Input: rectangular matrix A, vector b</br>
  Output: vector x 

  Test: residual || Ax-b ||
  </br>

# **Method**

Here the code for the assignment is provided.

###Sparse matrix-vector product
Function 1 is 'sparse matrix-vector product'

Given a vector $x$ and a sparse, real, quadratic matrix $A$ represented by CRS arrays val, col_idx, and row_ptr, this function returns $b = Ax$. It is also assumed that correct input is provided to the function.

In [14]:
def sparse_matrix_vector_product(x, val, col_idx, row_ptr):

  b = np.zeros(x.shape[0])
  for i in range(x.shape[0]):
    for j in range(row_ptr[i], row_ptr[i+1], 1): # index by zero in row_ptr and col_idx
      b[i] += val[j]*x[col_idx[j]]
  return b

To test the above code, we can run the following test function:

In [82]:
def test_sparse_matrix_vector_product():
  print("Testing spare_matrix_vector_product()")
  dim = 20
  a = sparse.random(dim, dim, density=0.4, format='csr', dtype='float')
  val = a.data
  col_idx = a.indices
  row_ptr = a.indptr
  x = np.random.random_sample([dim,])
  test = sparse_matrix_vector_product(x, val, col_idx, row_ptr)
  control = np.dot(a.todense(), x)
  assert np.allclose(test, control) == True, "incorrect result for sparse matrix-vector product"

if __name__ == '__main__':
  test_sparse_matrix_vector_product()

Testing spare_matrix_vector_product()


###QR factorization
Function 2 is 'QR factorization'

Given a real, quadratic matrix $A$, this function returns an orthogonal matrix $Q$ and an upper triangular matrix $R$, such that $A = QR$ 

In [16]:
def qr_factorization(a):
  if a.ndim != 2:
      return "error: a is not a matrix"
  if a.shape[0] != a.shape[1]:
      return "error: a is not a square matrix"
  r = np.zeros(a.shape)
  q = np.zeros(a.shape)
  for j in range(a.shape[0]):
    v = a[:, j]
    for i in range(j):
      r[i,j] = np.dot(q[:, i], v)
      v = np.subtract(v, r[i, j]*q[:, i])
    r[j, j] = np.linalg.norm(v)
    q[:, j] = v * (1 / r[j, j])
  return q, r

To test the above code, we can run the following test function:

In [17]:
def test_qr_factorization():
  print("Testing qr_factorization()")
  a = np.random.randint(10, size=(5, 5))
  test_q, test_r = qr_factorization(a)
  control = np.dot(test_q, test_r)
  assert np.allclose(a, control) == True, "incorrect result for qr factorization"
  # check if r is an upper triangular matrix
  tri = True
  for i in range(test_r.shape[0]):
    for j in range(0, i - 1):
      if test_r[i, j] != 0:
        tri = False
  assert tri, "r is not a triangular matrix"
  # Calculate || Q^TQ-I ||_F
  sum1 = 0
  qt_q = np.dot(np.transpose(test_q), test_q)
  id_m = np.identity(a.shape[0])
  for i in range(a.shape[0]):
    for j in range(a.shape[0]):
      s = qt_q[i, j] - id_m[i,j]
      sum1 += s*s
  frob1 =  math.sqrt(sum1)
  assert np.isclose(0, frob1), "frobenius norm || Q^TQ-I ||_F is not very small"
  # Calculate || QR-A ||_F
  sum2 = 0
  for i in range(a.shape[0]):
    for j in range(a.shape[0]):
      s = control[i, j] - a[i, j]
      sum2 += s*s
  frob2 =  math.sqrt(sum2)
  assert np.isclose(0, frob2), "frobenius norm || QR-A ||_F is not very small" 

This test function will check if $A = QR$, as well as if $Q$ is orthogonal (i.e. the distance between $Q^T Q$ and $I$ (identity matrix) is small), and if the distance between $QR$ and $A$ is small.

###Direct solver Ax=b
Function 3 is 'direct solver Ax=b'

Given a real, quadratic matrix $A$, and a vector $b$, this function returns the solution to the linear equation system $Ax = b$,  $x = A^{-1}b$. This is done by QR factorization of $A$ into $A = QR$, computation of $y = Q^T b$, and then solving $Rx = y$ by back substitution.

In [18]:
def direct_solve(a, b):
  q, r = qr_factorization(a)
  y = np.dot(np.transpose(q), b)
  n = a.shape[0]
  x = np.zeros(n)
  x[n - 1] = y[n - 1]/r[n - 1, n - 1]
  for i in range(n - 2, -1, -1):
    sum = 0
    for j in range(i + 1, n):
      sum += r[i, j]*x[j]
    x[i] = (y[i] - sum)/r[i, i]
  return x

To test the above code, we can run the following test function:

In [19]:
def test_direct_solve():
  print("Testing direct_solve()")
  a = np.random.randint(10, size=(10, 10))
  b = np.random.randint(10, size=10)
  x = direct_solve(a, b)
  residual = np.linalg.norm(np.subtract(np.dot(a, x), b))
  assert np.isclose(0, residual) == True, "incorrect result for direct solve"
  man_a = np.zeros((3,3))
  man_a[0, :] = [5, 3, 2]
  man_a[1, :] = [1, 1, 1]
  man_a[2, :] = [1, 3, 2]
  #matrix([[5, 3, 2], [1, 1, 1], [1, 3, 2]])
  man_b = np.array([60, 16, 36])
  man_x = np.matrix([6, 10, 0])
  y = direct_solve(man_a, man_b)
  assert np.isclose(0, np.linalg.norm(np.subtract(man_x, y))) == True, "distance to manufactured solution"

###Least squares problem Ax = b
Function 4 is 'least squares problem Ax = b'

Given a rectangular matrix $A$, and a vector $b$, this function returns an approximation of the solution to the linear equation system $Ax = b$.

In [20]:
def least_squares(a, b):
  sq_a = np.dot(np.transpose(a), a)
  new_b = np.dot(np.transpose(a), b)
  return direct_solve(sq_a, new_b)


To test the above code, we can run the following test function:

In [21]:
def test_least_squares():
  print("Testing least_squares()")
  sum = 0
  k = 100
  m = 20
  n = 5
  for i in range(k):
    a = np.random.randint(10, size=(m, n))
    b = np.random.randint(10, size=m)
    x = least_squares(a, b)
    sum += np.linalg.norm(np.subtract(np.dot(a, x), b))
  sum = sum/100
  print(f'Least squares method produced a mean norm of {sum} over {k} tests of {m}*{n}-matrices (A), and {m}-vectors (b)')


###Testing
Then to perform all the tests, we can run the following code:

In [81]:
def run_all_tests():
  test_sparse_matrix_vector_product()
  test_qr_factorization()
  test_direct_solve()
  test_least_squares()
  print("All tests OK")

if __name__ == '__main__':
  run_all_tests()

Testing spare_matrix_vector_product()
Testing qr_factorization()
Testing direct_solve()
Testing least_squares()
Least squares method produced a mean norm of 11.847098864779085 over 100 tests of 20*5-matrices (A), and 20-vectors (b)
All tests OK


# **Results**

Running the test cases here in google colab, after importing required libraries, defining all functions, etc., generates the following output:
```
Testing spare_matrix_vector_product()
Testing qr_factorization()
Testing direct_solve()
Testing least_squares()
Least squares method produced a mean norm of 12.083311383755506 over 100 tests of 20*5-matrices (A), and 20-vectors (b)
All tests OK
```
From which we can see that all test cases were passed, as well as the performance of the least squares method implementation.

# **Discussion**

The functions have the correct input and output as required by the lab instructions, as verified by the test cases.

I was not sure what the required margin of error would be for the least squares method, and I did not implement an extensive test case suite for plotting performance.